# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

Using TensorFlow backend.


In [0]:
X_train1=[]
y_train1=[]
X_test1=[]
y_test1=[]
X_train2=[]
y_train2=[]
X_test2=[]
y_test2=[]
for idx,x in enumerate(y_train):
    if(x<5):
        X_train1.append(X_train[idx])
        y_train1.append(y_train[idx])
    else:
        X_train2.append(X_train[idx])
        y_train2.append(y_train[idx])
for idx,x in enumerate(y_test):
    if(x<5):
        X_test1.append(X_test[idx])
        y_test1.append(y_test[idx])
    else:
        X_test2.append(X_test[idx])
        y_test2.append(y_test[idx])
import numpy as np
X_train1=np.asarray(X_train1)
y_train1=np.asarray(y_train1)
X_test1=np.asarray(X_test1)
y_test1=np.asarray(y_test1)
X_train2=np.asarray(X_train2)
y_train2=np.asarray(y_train2)
X_test2=np.asarray(X_test2)
y_test2=np.asarray(y_test2)

## splitting the dataset based on target value, 0-4 in one group and 5-9 in another group

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
import tensorflow as tf
y_train1 = tf.keras.utils.to_categorical(y_train1, num_classes = 5)
y_test1 = tf.keras.utils.to_categorical(y_test1, num_classes = 5)
X_train1 = X_train1/255.0
X_test1 = X_test1/255.0

## encoding the Y target and standardization of the X variables 

In [4]:
X_train1.shape

(25000, 32, 32, 3)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [5]:
model=tf.keras.Sequential()  ##Conv layer
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3),strides =(1,1),activation='relu',input_shape=(32,32,3),name='fc1'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3),strides =(1,1),activation='relu',name='fc2'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides =(2,2), padding='valid',name='fc3'))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3),strides =(1,1),activation='relu',name='fc4'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3),strides =(1,1),activation='relu',name='fc5'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides =(2,2), padding='valid',name='fc6'))
model.add(tf.keras.layers.Dropout(0.25))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.add(tf.keras.layers.Flatten()) ## Deep neural network
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.LayerNormalization())
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(5, activation='softmax'))

In [7]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Conv2D)                 (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 32)        128       
_________________________________________________________________
fc2 (Conv2D)                 (None, 28, 28, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
fc3 (MaxPooling2D)           (None, 14, 14, 32)        0         
_________________________________________________________________
fc4 (Conv2D)                 (None, 12, 12, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 12, 12, 32)        1

In [8]:
model.fit(X_train1,y_train1,batch_size=100,epochs=15,validation_data=(X_test1,y_test1))

Train on 25000 samples, validate on 5000 samples
Epoch 1/15
25000/25000 [==============================] - 100s 4ms/sample - loss: 1.0948 - acc: 0.5641 - val_loss: 1.9528 - val_acc: 0.3438
Epoch 2/15
25000/25000 [==============================] - 101s 4ms/sample - loss: 0.8069 - acc: 0.6876 - val_loss: 1.7306 - val_acc: 0.4180
Epoch 3/15
25000/25000 [==============================] - 101s 4ms/sample - loss: 0.6839 - acc: 0.7435 - val_loss: 0.8498 - val_acc: 0.6906
Epoch 4/15
25000/25000 [==============================] - 101s 4ms/sample - loss: 0.5947 - acc: 0.7778 - val_loss: 0.7036 - val_acc: 0.7470
Epoch 5/15
25000/25000 [==============================] - 100s 4ms/sample - loss: 0.5404 - acc: 0.7994 - val_loss: 0.5730 - val_acc: 0.7834
Epoch 6/15
25000/25000 [==============================] - 101s 4ms/sample - loss: 0.4936 - acc: 0.8205 - val_loss: 0.6010 - val_acc: 0.7790
Epoch 7/15
25000/25000 [==============================] - 100s 4ms/sample - loss: 0.4473 - acc: 0.8358 - val_lo

In [9]:
scores = model.evaluate(X_train1,y_train1, verbose=0)
print("Train Accuracy: %.2f%%" % (scores[1]*100))
scores = model.evaluate(X_test1,y_test1, verbose=0)
print("Test Accuracy: %.2f%%" % (scores[1]*100))

Train Accuracy: 92.08%
Test Accuracy: 81.90%


In [0]:
y_test11=[]
for idx,x in enumerate(y_test):
    if(x<5):
        y_test11.append(y_test[idx])
y_test11=np.asarray(y_test11)

In [11]:
from sklearn.metrics import classification_report
print("Classification Report")
print(classification_report(y_test11,model.predict_classes(X_test1)))

#model is performing good on the whole with 80+ avg accuracy on the test set 

Classification Report
              precision    recall  f1-score   support

           0       0.80      0.90      0.85      1000
           1       0.93      0.96      0.95      1000
           2       0.77      0.69      0.73      1000
           3       0.78      0.74      0.76      1000
           4       0.80      0.81      0.80      1000

    accuracy                           0.82      5000
   macro avg       0.82      0.82      0.82      5000
weighted avg       0.82      0.82      0.82      5000



### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [12]:
for layer in model.layers:
    if layer.name == 'fc1' or layer.name == 'fc2' or layer.name == 'fc3' or layer.name == 'fc4' or layer.name == 'fc5' or layer.name == 'fc6':
        layer.trainable = False

## hardcoding the conv layers fc1-6 as non trainable

from termcolor import colored    #  python module for ANSII Color formatting for output in terminal
#Check which layers have been frozen 
for layer in model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

fc1
False
batch_normalization
True
fc2
False
batch_normalization_1
True
fc3
False
fc4
False
batch_normalization_2
True
fc5
False
batch_normalization_3
True
fc6
False
dropout
True
flatten
True
dense
True
dense_1
True
layer_normalization
True
dropout_1
True
dense_2
True


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
y_train2=y_train2-5
y_test2=y_test2-5
X_train2 = X_train2/255.0
X_test2 = X_test2/255.0
y_train2 = tf.keras.utils.to_categorical(y_train2, num_classes = 5)
y_test2 = tf.keras.utils.to_categorical(y_test2, num_classes = 5)

## encoding the Y target and standardization of the X variables 

In [14]:
model.fit(X_train2,y_train2,batch_size=100,epochs=5,validation_data=(X_test2,y_test2))

## In this, Conv layers weights are not modified and only deep neural network weights are adjusted. (Transfer learning) 

Train on 25000 samples, validate on 5000 samples
Epoch 1/5
25000/25000 [==============================] - 99s 4ms/sample - loss: 0.7174 - acc: 0.7619 - val_loss: 0.4848 - val_acc: 0.8232
Epoch 2/5
25000/25000 [==============================] - 100s 4ms/sample - loss: 0.3992 - acc: 0.8599 - val_loss: 0.4319 - val_acc: 0.8458
Epoch 3/5
25000/25000 [==============================] - 100s 4ms/sample - loss: 0.3346 - acc: 0.8817 - val_loss: 0.3575 - val_acc: 0.8718
Epoch 4/5
25000/25000 [==============================] - 100s 4ms/sample - loss: 0.2906 - acc: 0.8968 - val_loss: 0.3659 - val_acc: 0.8744
Epoch 5/5
25000/25000 [==============================] - 99s 4ms/sample - loss: 0.2521 - acc: 0.9100 - val_loss: 0.3038 - val_acc: 0.8934


In [15]:
scores = model.evaluate(X_train2,y_train2, verbose=0)
print("Train Accuracy: %.2f%%" % (scores[1]*100))
scores = model.evaluate(X_test2,y_test2, verbose=0)
print("Test Accuracy: %.2f%%" % (scores[1]*100))

Train Accuracy: 93.68%
Test Accuracy: 89.34%


In [0]:
y_test22=[]
for idx,x in enumerate(y_test):
    if(x>4):
        y_test22.append(y_test[idx])
y_test22=np.asarray(y_test22)
y_test22=y_test22-5

In [17]:
print("Classification Report")
print(classification_report(y_test22,model.predict_classes(X_test2)))

## Model performs good on the whole, with 85%+ avg accuracy on the test set. 

Classification Report
              precision    recall  f1-score   support

           0       0.83      0.87      0.85      1000
           1       0.90      0.93      0.91      1000
           2       0.88      0.84      0.86      1000
           3       0.93      0.93      0.93      1000
           4       0.94      0.91      0.92      1000

    accuracy                           0.89      5000
   macro avg       0.89      0.89      0.89      5000
weighted avg       0.89      0.89      0.89      5000



# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [20]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [22]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [23]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [24]:
twenty_train.data[1:5]

["From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the problem:\n\tI have a rectangular mesh in the uv domain, i.e  the mesh is a \n\tmapping of a 3d Bezier patch into 2d. The area in this domain\n\twhich is inside a trimming loop had to be rendered. The trimming\n\tloop is a set of 2d Bezier curve segments.\n\tFor the sake of notation: the mesh is made up of cells.\n\n\tMy problem is this :\n\tThe trimming area has to be split up into individual smaller\n\tcells bounded by the trimming curve segments. If a cell\n\tis wholly inside the area...then it is output as a whole ,\n\telse it is trivially rejected. \n\n\tDoes any body know how thiss can be done, or is there any algo. \n\tsomewhere for doing this.\n\n\tAny help would be appreciated.\n\n\tThanks, \n\tAni.

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = CountVectorizer(stop_words='english',ngram_range=(1,2),min_df=2)
X_train_dtm = vect.fit_transform(twenty_train.data)
X_test_dtm = vect.transform(twenty_test.data)

##Creating Tf-idf for both Test and Train with standard ENGLISH stop words and ngram range of (1,2) and min_df as 2

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()

In [27]:
from sklearn import metrics
lg.fit(X_train_dtm, twenty_train.target)

##fitting the data to logistic regression model


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [28]:
y_pred_class = lg.predict(X_train_dtm)
print('Train Accuracy: ', metrics.accuracy_score(twenty_train.target, y_pred_class))

Train Accuracy:  1.0


In [29]:
y_pred_class = lg.predict(X_test_dtm)
print('Test Accuracy: ', metrics.accuracy_score(twenty_test.target, y_pred_class))

## we have a model with 90%+ accuracy on the test set.

Test Accuracy:  0.9041278295605859
